# Testing to get all liked songs of a user

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
from datetime import datetime

In [2]:
#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials('bf1ba68423404778a60bcf3dee58d199','7365dc611a2d4ddba4ad61343f0b64d7'))

sp = spotipy.Spotify(auth_manager=SpotifyOAuth('86677c795a49463d9281fac012a87155','fe6f941da771447c920e02bbb2a82859', redirect_uri='http://localhost:5000',scope='user-library-read') )


In [3]:

user = sp.user('nobodyknowsmynam')
print(user)

{'display_name': 'Matt Laraia', 'external_urls': {'spotify': 'https://open.spotify.com/user/nobodyknowsmynam'}, 'followers': {'href': None, 'total': 9}, 'href': 'https://api.spotify.com/v1/users/nobodyknowsmynam', 'id': 'nobodyknowsmynam', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab6775700000ee854f8dbba7048b2fbd494b14df', 'width': None}], 'type': 'user', 'uri': 'spotify:user:nobodyknowsmynam'}


In [4]:
myPlaylists = sp.user_playlists('nobodyknowsmynam')

In [5]:
len(myPlaylists)

7

In [9]:
type(myPlaylists['items'])

list

In [10]:
for item in myPlaylists['items']:
    print(item['name'])

I guess so
raaage
Relax
Summer
Meditate
Positivity 😃
YES
H I T S
Jazz Vibes
Hype
Deep Focus
Songs to Sing in the Car


# Results:

Using a token, redirect URI, and scope, we can get the user's liked songs and maybe even extra songs they didn't like but put in their playlists.

In [11]:
sp.current_user_saved_tracks()

{'href': 'https://api.spotify.com/v1/me/tracks?offset=0&limit=20',
 'items': [{'added_at': '2023-03-24T16:11:54Z',
   'track': {'album': {'album_group': 'single',
     'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1grN0519h2zYqpRtYbDZAl'},
       'href': 'https://api.spotify.com/v1/artists/1grN0519h2zYqpRtYbDZAl',
       'id': '1grN0519h2zYqpRtYbDZAl',
       'name': 'Larry June',
       'type': 'artist',
       'uri': 'spotify:artist:1grN0519h2zYqpRtYbDZAl'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/0eVyjRhzZKke2KFYTcDkeu'},
       'href': 'https://api.spotify.com/v1/artists/0eVyjRhzZKke2KFYTcDkeu',
       'id': '0eVyjRhzZKke2KFYTcDkeu',
       'name': 'The Alchemist',
       'type': 'artist',
       'uri': 'spotify:artist:0eVyjRhzZKke2KFYTcDkeu'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/0c173mlxpT3dSFRgMO8XPh'},
       'href': 'https://api.spotify.com/v1/artists/0c173mlx

# Batch Processing of User's Liked Songs

In [23]:
track1=sp.current_user_saved_tracks(limit=30)['items'][1]['track']['id']

In [25]:
track2=sp.current_user_saved_tracks(limit=30)['items'][2]['track']['id']

In [74]:
def getRelevantTrackFeatures(sp,user_prior_login_date=''):
    track_ids = retrieveTrackIds(sp, user_prior_login_date)
    track_ids = checkIdsInDatabase(track_ids)
    features = retrieveTrackFeatures(sp, track_ids)
    saveTrackFeaturesToDatabase(features)
    updateLastLoginDate(sp)
    return features

def retrieveTrackIds(sp, user_prior_login_date):
    track_ids = []
    offset = 0
    limit = 50
    liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)
    endLoopEarly = False

    while True:
        for item in liked_tracks['items']:
            if user_prior_login_date != '':
                if item['added_at']> user_prior_login_date:
                    track_ids.append(item['track']['id'])
                else:
                    endLoopEarly = True
                    break
            else:
                track_ids.append(item['track']['id'])
        offset += limit
        
        if len(liked_tracks['items']) < limit or endLoopEarly:
            # All tracks have been retrieved
            break
        
        liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)

    return track_ids

def checkIdsInDatabase(track_ids):
    #check if the track_ids are already in the database
    #if they are, remove them from the track_ids list
    for track_id in track_ids:
        if track_id in database:
            track_ids.remove(track_id)
    return track_ids



def retrieveTrackFeatures(sp, track_ids):
    features = []
    for i in range(0, len(track_ids), 50):
        features += sp.audio_features(track_ids[i:i+50])
        #only get the features we need, so we do not need type, uri, analysis_url, track_href
        for feature in features:
            del feature['type']
            del feature['uri']
            del feature['analysis_url']
            del feature['track_href']
    return features

def saveTrackFeaturesToDatabase(features):
    #save the features to the database
    for feature in features:
        database[feature['id']] = feature
    return database

def updateLastLoginDate(sp):
    #update the last login date in the database
    database['last_login_date'] = datetime.now()
    return 

In [58]:
track_features = retrieveTrackFeatures(sp, track_ids[:51])

In [60]:
track_features[0]

{'danceability': 0.637,
 'energy': 0.58,
 'key': 2,
 'loudness': -7.862,
 'mode': 1,
 'speechiness': 0.0512,
 'acousticness': 0.0951,
 'instrumentalness': 0,
 'liveness': 0.11,
 'valence': 0.514,
 'tempo': 78.509,
 'type': 'audio_features',
 'id': '2YALU0elzWXiee1nhHSWR6',
 'uri': 'spotify:track:2YALU0elzWXiee1nhHSWR6',
 'track_href': 'https://api.spotify.com/v1/tracks/2YALU0elzWXiee1nhHSWR6',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2YALU0elzWXiee1nhHSWR6',
 'duration_ms': 220127,
 'time_signature': 4}

In [51]:
print (len(track_ids))

3719


In [44]:
track_ids[0]

'7ijXfR09g8dImpTI586m7k'

In [57]:
len(track_ids)

3719

In [46]:
#get the audio features of track1
len(sp.audio_features(track_ids))

50

In [64]:
test_sample = sp.current_user_saved_tracks(limit=5, offset=50)

In [66]:
test_sample['items'][0]['added_at']

'2023-02-05T16:21:55Z'

In [69]:

date_str = test_sample['items'][0]['added_at']
dt = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
dt

datetime.datetime(2023, 2, 5, 16, 21, 55)

In [70]:
most_recent = sp.current_user_saved_tracks(limit=1)

In [72]:
recent_dt_str=most_recent['items'][0]['added_at']
recent_dt = datetime.strptime(recent_dt_str, '%Y-%m-%dT%H:%M:%SZ')
recent_dt

datetime.datetime(2023, 3, 24, 16, 11, 54)

In [73]:
recent_dt>dt

True